In [1]:
using TuringModels

Turing.setadbackend(:reverse_diff);
Turing.turnprogress(false);

d = CSV.read(rel_path("..", "data", "Kline.csv"), delim=';');
size(d) # Should be 10x5

┌ Info: [Turing]: global PROGRESS is set as false
└ @ Turing /Users/rob/.julia/dev/Turing/src/Turing.jl:24


(10, 5)

New col log_pop, set log() for population data

In [2]:
d[:log_pop] = map(x -> log(x), d[:population]);

New col contact_high, set binary values 1/0 if high/low contact

In [3]:
d[:contact_high] = map(x -> ifelse(x=="high", 1, 0), d[:contact]);

This is supposed to be a "bad" model since we take non-centered data for the
predictor log_pop

In [4]:
@model m10_10stan(total_tools, log_pop, contact_high) = begin
    α ~ Normal(0, 100)
    βp ~ Normal(0, 1)
    βc ~ Normal(0, 1)
    βpc ~ Normal(0, 1)

    for i ∈ 1:length(total_tools)
        λ = exp(α + βp*log_pop[i] + βc*contact_high[i] +
            βpc*contact_high[i]*log_pop[i])
        total_tools[i] ~ Poisson(λ)
    end
end;

posterior = sample(m10_10stan(d[:total_tools], d[:log_pop],
    d[:contact_high]), Turing.NUTS(2000, 1000, 0.95));
# Fix the inclusion of adaptation samples
posterior2 = MCMCChain.Chains(posterior.value[1001:2000,:,:], names=posterior.names);

┌ Info: [Turing] looking for good initial eps...
└ @ Turing.Inference /Users/rob/.julia/dev/Turing/src/inference/support/hmc_core.jl:240
┌ Warning: Numerical error has been found in gradients.
└ @ Turing.Core /Users/rob/.julia/dev/Turing/src/core/ad.jl:168
┌ Warning: grad = [NaN, NaN, NaN, NaN]
└ @ Turing.Core /Users/rob/.julia/dev/Turing/src/core/ad.jl:169
┌ Warning: Numerical error has been found in gradients.
└ @ Turing.Core /Users/rob/.julia/dev/Turing/src/core/ad.jl:168
┌ Warning: grad = [NaN, NaN, NaN, NaN]
└ @ Turing.Core /Users/rob/.julia/dev/Turing/src/core/ad.jl:169
┌ Warning: Numerical error has been found in gradients.
└ @ Turing.Core /Users/rob/.julia/dev/Turing/src/core/ad.jl:168
┌ Warning: grad = [NaN, NaN, NaN, NaN]
└ @ Turing.Core /Users/rob/.julia/dev/Turing/src/core/ad.jl:169
┌ Warning: Numerical error has been found in gradients.
└ @ Turing.Core /Users/rob/.julia/dev/Turing/src/core/ad.jl:168
┌ Warning: grad = [NaN, NaN, NaN, NaN]
└ @ Turing.Core /Users/rob/.julia/d

Rethinking result

In [5]:
m_10_10t_result = "
     mean   sd  5.5% 94.5% n_eff Rhat
 a    0.94 0.37  0.36  1.53  3379    1
 bp   0.26 0.04  0.21  0.32  3347    1
 bc  -0.08 0.84 -1.41  1.23  2950    1
 bpc  0.04 0.09 -0.10  0.19  2907    1
";

Describe the draws

In [6]:
describe(posterior2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1
Samples per chain = 1000

Empirical Posterior Estimates:
              Mean                  SD                        Naive SE                MCSE         ESS    
       α   0.918178947  0.3460359900265679855557721 0.01094261880875264068835317 0.06666126422   26.946071
  lf_num   0.000000000  0.0000000000000000000000000 0.00000000000000000000000000 0.00000000000         NaN
      βc  -0.017650561  0.8515630480330687035461779 0.02692878803019865818768608 0.23227427157   13.440994
     βpc   0.035631581  0.0927513371350874721299462 0.00293305481373032882197460 0.02491304271   13.860753
 elapsed   0.023869051  0.0133581401367446582495457 0.00042242148135826214187805 0.00062899789  451.017727
 epsilon   0.003251720  0.0000000000000000017355915 0.00000000000000000005488422 0.00000000000 1000.000000
      βp   0.265469649  0.0334668486927697100830059 0.00105831467977380981083935 0.00622322475   28.920025
      lp -45.812021553  1.48039153

End of m_10_10t.jl

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*